# European universities

## Data preparation

### Filtering the dataset

This script will filter the University rankings dataset by the country of origin. In the .csv file 00_eu_country_codes.csv we defined all the European country ISO codes. Using those codes we can easily perform the filtering.


In [1]:
import pandas as pd

df = pd.read_csv("../datasets/00_eu_country_codes.csv")
country_codes = df["Country codes"].tolist()

df = pd.read_csv("../datasets/00_2024_QS_World_university_rankings.csv")
mask = df["Country Code"].isin(country_codes)

filtered_df = df[mask]

filtered_df.to_csv("../datasets/01_Universities_ranking_filtered.csv", index=False)

print("Filtering finished.")


Filtering finished.


## Enriching data

Next we are going to start fetching the data for each university with different methods which will be explained as we go. However, the datasets we will be using are:

- 2024 University ranking (.csv file from Kaggle - used in **Filtering the dataset**)
- Wikipedia API to get the plain text of the Wikipedia page of that university
- WikiData API to get general university data 
- WikiData API to get the cities where the universities are located 
- Wikipedia API to get the plain text of the Wikipedia page of that city

However, before doing so, we still need to adjust the current dataset. Since some university names are written in their local language, we need to manually translate them to english, since we will be using the English Wikipedia and Wikidata API. Starting from the file 01_Universities_ranking_filtered.csv we get in the last step, we can create 01_Universities_ranking_filtered_renamed.csv.

For this purpose we renamed the old "Institution Name" to "Institution Name - WRONG" and created a new column with the heading "Institution Name" where we added the correct University names.


### University Wikidata

This script reads the english university names and fetches data for them from the Wikidata API. The most important thing we get with it is the ID of the actual Wikipedia page as well as confirmation that the Wikipedia page for that university exists.

In [4]:
import requests
from pandas import *
import json

base_url = "https://wikidata.org/w/api.php?action=wbgetentities&sites=enwiki&props=descriptions&languages=en&format=json"
csv_file_path = "../datasets/01_Universities_ranking_filtered_renamed.csv"
new_csv_file_path = "../datasets/02_Universities_with_wikidata.csv"

# Prepare dataset
data = read_csv(csv_file_path)
institution_name = data['Institution Name'].tolist()

# Results
results = []

for index, i in enumerate(institution_name):
    # fetch data for University
    try:
        print("Starting API call for: ", index, " ", i)
        url = base_url + "&titles=" + institution_name[index].replace(" ", "_")
        r = requests.get(url)
        request_payload = json.loads(r.content.decode())

        # check if university data is found and update the field
        if ("-1" not in request_payload["entities"].keys()):
            data.loc[index, 'Wikidata'] = (json.dumps(request_payload))
            print("Finished API call for: ", index, " ", i, " - OK")
        else:
            results.append(False)
            data.loc[index, 'Wikidata'] = "NOT FOUND"
            print("Finished API call for: ", index, " ", i, " - NOT FOUND")
    except Exception as ex:
        print("Failed API call for: ", index, " ", i, " with exception: ", ex)
        data.loc[index, "Wikidata"] = "ERROR"

# writing into the file
data.to_csv(new_csv_file_path, index=False)

print("Url don't exist count: " + str(results.count(False)))


Starting API call for:  0   University of Cambridge
Finished API call for:  0   University of Cambridge  - OK
Starting API call for:  1   University of Oxford
Finished API call for:  1   University of Oxford  - OK
Starting API call for:  2   Imperial College London
Finished API call for:  2   Imperial College London  - OK
Starting API call for:  3   ETH Zurich
Finished API call for:  3   ETH Zurich  - OK
Starting API call for:  4   UCL
Finished API call for:  4   UCL  - OK
Starting API call for:  5   University of Edinburgh
Finished API call for:  5   University of Edinburgh  - OK
Starting API call for:  6   Paris Sciences et Lettres University
Finished API call for:  6   Paris Sciences et Lettres University  - OK
Starting API call for:  7   University of Manchester
Finished API call for:  7   University of Manchester  - OK
Starting API call for:  8   École Polytechnique Fédérale de Lausanne
Finished API call for:  8   École Polytechnique Fédérale de Lausanne  - OK
Starting API call fo

**RESULTS:** Only 1 universities was not found, because it had the character & in it's name (Wageningen University & Research). Because of that we did it manually by changing the & with %26.

### University Wikipedia plain text

This script reads the university's id from the Wikidata fetched in the last code chunk, then it fetches the Wikipedia page content by it's title, and finally parses the wikitext to plain text with the mwparserfromhell libray.

In [21]:
import requests
import mwparserfromhell
import pandas as pd

csv_file_path = "../datasets/02_Universities_with_wikidata.csv"
new_csv_file_path = "../datasets/03_Universities_with_wikipedia_page.csv"
base_url = "https://en.wikipedia.org/w/api.php"

# Prepare dataset
data = pd.read_csv(csv_file_path)
wikidata_list = data['Wikidata'].tolist()

skipped = 0

for index, wikidata in enumerate(wikidata_list):
    try:
        print("Retrieving wikipedia text for: ", index)
        
        wikidata_id = list(json.loads(wikidata)['entities'].keys())[0]

        # Get Wikipedia Page ID from Wikidata ID
        wikidata_url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={wikidata_id}&format=json"
        r = requests.get(wikidata_url)
        wikidata_data = r.json()
        title = wikidata_data['entities'][wikidata_id]['sitelinks']['enwiki']['title']
        print("Retrieving wikipedia text for: ", index, " ", title)

        # Get Wikipedia Page Text using Page ID
        wikipedia_url = f"https://en.wikipedia.org/w/api.php?action=query&titles={title}&prop=revisions&rvprop=content&format=json"
        r = requests.get(wikipedia_url)
        wikipedia_data = r.json()
        page_id = list(wikipedia_data['query']['pages'].keys())[0]
        page_text = wikipedia_data['query']['pages'][page_id]['revisions'][0]['*']

        # page_text is in wikitext format, we have to convert it to plain text
        parsed_wikitext = mwparserfromhell.parse(page_text)
        plain_text = parsed_wikitext.strip_code()
        data.loc[index, 'Wikipedia Text'] = plain_text
    except Exception as ex: 
        print("Failed API call for: ", index, " with exception: ", ex)
        data.loc[index, "Wikipedia Text"] = "ERROR"
        skipped += 1

#write to file
data.to_csv(new_csv_file_path, index=False)
print("Wikipedia text NOT FOUND for: ", skipped, " universities.")



Retrieving wikipedia text for:  0
Retrieving wikipedia text for:  0   University of Cambridge
Retrieving wikipedia text for:  1
Retrieving wikipedia text for:  1   University of Oxford
Retrieving wikipedia text for:  2
Retrieving wikipedia text for:  2   Imperial College London
Retrieving wikipedia text for:  3
Retrieving wikipedia text for:  3   ETH Zurich
Retrieving wikipedia text for:  4
Retrieving wikipedia text for:  4   UCL
Retrieving wikipedia text for:  5
Retrieving wikipedia text for:  5   University of Edinburgh
Retrieving wikipedia text for:  6
Retrieving wikipedia text for:  6   Paris Sciences et Lettres University
Retrieving wikipedia text for:  7
Retrieving wikipedia text for:  7   University of Manchester
Retrieving wikipedia text for:  8
Retrieving wikipedia text for:  8   École Polytechnique Fédérale de Lausanne
Retrieving wikipedia text for:  9
Retrieving wikipedia text for:  9   Technical University of Munich
Retrieving wikipedia text for:  10
Retrieving wikipedia te

**RESULTS:** All university Wikipedia pages were retrieved successfully.

### University cities wikipedia plaintext

This script reads from the WikiData data and searches for references for the city in which the university is based. 

It searches for one of the following relationships:
- located in the administrative territorial entity P131

If it finds a relationship it will use the same logic for fetching Wikipedia page plain text as before, for that city.

In [25]:
import requests
from pandas import *
import json

base_url = "https://en.wikipedia.org/w/api.php"
wikidata_base_url = "https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/"
csv_file_path = "../datasets/03_Universities_with_wikipedia_page.csv"
new_csv_file_path = "../datasets/04_Universities_with_city_wikipedia_page.csv"

# Prepare dataset
data = read_csv(csv_file_path)
university_wikidata = data['Wikidata'].tolist()

# Results
results = []

for index, uni_wikidata in enumerate(university_wikidata):
    try:
        print("Starting API call for: ", index)

        # get University wikidata from file
        wikidata_json = json.loads(uni_wikidata)
        wikidata_json = wikidata_json["entities"]
        university_id = list(wikidata_json.keys())[0]

        # Get the whole wikidata page for university
        response_body = requests.get(
            wikidata_base_url + university_id,
        ).json()

        university_relations = response_body["statements"]
        university_relations_keys = list(university_relations.keys())

        # Get the city wikidata if the relationship to the city exists
        # TODO check what other relations there are because some dont have this one, but have like location
        
        possible_relations_list = ["P131", "P276"]
        relation_found = False
        
        for rel in possible_relations_list:
            if (rel in university_relations_keys):
                relation_found = True
                
                city_id = university_relations[rel][0]["value"]["content"]
                print("Found city - Starting API call for: ", city_id)

                city_response_body = requests.get(
                    wikidata_base_url + city_id,
                ).json()

                city_name = city_response_body["labels"]["en"]
                print("Found city wikidata - Starting API call for: ", city_name)

                # Get Wikipedia Page Text using the city's title
                wikipedia_url = f"https://en.wikipedia.org/w/api.php?action=query&titles={city_name}&prop=revisions&rvprop=content&format=json"
                r = requests.get(wikipedia_url)
                wikipedia_data = r.json()
                page_id = list(wikipedia_data['query']['pages'].keys())[0]
                page_text = wikipedia_data['query']['pages'][page_id]['revisions'][0]['*']

                # page_text is in wikitext format, we have to convert it to plain text
                parsed_wikitext = mwparserfromhell.parse(page_text)
                plain_text = parsed_wikitext.strip_code()
                data.loc[index, 'City wikipedia text'] = plain_text
            
        if relation_found is False:
            results.append(False)
            data.loc[index, 'City wikipedia text'] = "NOT FOUND"
            print("Finished API call for: ", index, " - NOT FOUND")
    except Exception as ex:
        print("Unexpected error for: ", index, " with exception: ", ex)
        data.loc[index, "City wikipedia text"] = "ERROR"

# writing into the file
data.to_csv(new_csv_file_path, index=False)

print("Url don't exist count: " + str(results.count(False)))


Starting API call for:  0
Found city - Starting API call for:  Q23112
Found city wikidata - Starting API call for:  Cambridgeshire
Found city - Starting API call for:  Q350
Found city wikidata - Starting API call for:  Cambridge
Starting API call for:  1
Found city - Starting API call for:  Q34217
Found city wikidata - Starting API call for:  Oxford
Found city - Starting API call for:  Q34217
Found city wikidata - Starting API call for:  Oxford
Starting API call for:  2
Found city - Starting API call for:  Q188801
Found city wikidata - Starting API call for:  Royal Borough of Kensington and Chelsea
Found city - Starting API call for:  Q86088281
Found city wikidata - Starting API call for:  South Kensington Campus, Imperial College London
Unexpected error for:  2  with exception:  'revisions'
Starting API call for:  3
Found city - Starting API call for:  Q72
Found city wikidata - Starting API call for:  Zürich
Found city - Starting API call for:  Q72
Found city wikidata - Starting API c

## Data analysis

Stats before cleaning data:
- Unis with at least one column missing:70

In [ ]:
raw_data_path = '../datasets/00_2024_QS_World_university_rankings.csv'
cleaned_data_path = '../datasets/04_Universities_with_city_wikipedia_page.csv'

In [ ]:
#Returns a list of universities and the columns that they are missing(missing value)
import pandas as pd

df = pd.read_csv(cleaned_data_path)

if 'Institution Name' in df.columns:
    #create a dictionary to store missing values for each university
    missing_values_by_uni = {}

    #iterate through rows and group missing values by uni
    for index, row in df.iterrows():
        university_name = row['Institution Name']
        missing_columns = [column for column in df.columns if column != 'Institution Name' and pd.isnull(row[column])]
        
        #store only universities that have a missing column
        if university_name not in missing_values_by_uni and len(missing_columns) > 0:
            missing_values_by_uni[university_name] = []
            missing_values_by_uni[university_name].extend(missing_columns)

    #print missing values grouped by university name
    for university, missing_columns in missing_values_by_uni.items():
        print(f"Missing values for {university} are: {', '.join(missing_columns)}")
    
    print("Number of universities that have at least one missing value:", len(missing_values_by_uni))
else:
    print("Error: 'Institution Name' column not found in the DataFrame.")

In [ ]:
#Get 5 columns that have most missing values:
#results using full_list.csv
from collections import Counter
columns = missing_values_by_uni.values()
result = []
for l in columns:
    result.extend(l)
    
most_common_missing_columns = Counter(result).most_common(5)
print([name for name,_ in most_common_missing_columns])
print(most_common_missing_columns)

In [ ]:
#Delete the 5 columns which have most missing values:
columns_to_delete = [name for name,_ in most_common_missing_columns]
df.drop(columns=columns_to_delete, inplace=True)
output_file_path = 'datasets/cleaned_full_list.csv'
df.to_csv(output_file_path, index=False)

In [ ]:
#Get 10 universities which have most missing values/columns
count_missing_values_by_uni = {}
for uni in missing_values_by_uni:
    count_missing_values_by_uni[uni] = len(missing_values_by_uni[uni])
top_10_unis = dict(sorted(count_missing_values_by_uni.items(), key=lambda item: item[1], reverse=True)[:10])
print(top_10_unis)

In [ ]:
#Delete the 10 unis
unis_name = [uni for uni in top_10_unis]
print(unis_name)
mask = ~df['Institution Name'].isin(unis_name)
filtered_data = df[mask]
filtered_data.to_csv(output_file_path, index = False)